In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
from plot_config import *

In [3]:
import os
path = '/sciserver/filedb10-01/ocean/wenrui_temp/table_domain5/'
lst = os.listdir(path)
print(len(os.listdir(path)))

slices = []
for i in range(5):
    slices.append(str(slice(i*730,(i+1)*730)))
slices

subs = []
offsets = [i*730 for i in range(5)]
for slid in range(5): 
    strslice = slices[slid]
    sub_name = ['table'+str(i)+strslice for i in range(0,161)]
    for name in sub_name:
        if name not in lst:
            print(name)
    datasets = [xr.open_zarr(path+name) for name in sub_name]
    
    sub = xr.concat(datasets,dim = 'time')

    sub['space']=sub.space+offsets[slid]
    subs.append(sub)

ds = xr.concat(subs,dim = 'space')

805


In [5]:
NUMBER_OF_PARTICLE = NUMBER_OF_PARTICLE_domain4

In [6]:
for a_region in region_names:
    ds[a_region] = xr.zeros_like(ds['A'])
    for var in ds.data_vars:
        if a_region in var:
            ds[a_region]+=ds[var]
            print(var, end = ' ')
    print()
ds['sum'] = xr.zeros_like(ds['A'])
for var in rhs_list:
    ds['sum']+=ds[var]
    print(var, end = ' ')

A_gulf E_gulf F_gulf I_gulf dif_h_gulf dif_v_gulf e_ua_gulf gulf 
A_labr E_labr F_labr I_labr dif_h_labr dif_v_labr e_ua_labr labr 
A_gdbk E_gdbk F_gdbk I_gdbk dif_h_gdbk dif_v_gdbk e_ua_gdbk gdbk 
A_nace E_nace F_nace I_nace dif_h_nace dif_v_nace e_ua_nace nace 
A_egrl E_egrl F_egrl I_egrl dif_h_egrl dif_v_egrl e_ua_egrl egrl 
e_ua E dif_h dif_v A I F 

In [7]:
ds

<xarray.Dataset>
Dimensions:     (time: 9497, space: 3649)
Coordinates:
  * time        (time) datetime64[ns] 1992-01-01T18:00:00 ... 2017-12-31T06:0...
  * space       (space) int64 0 1 2 3 4 5 6 ... 3643 3644 3645 3646 3647 3648
Data variables: (12/51)
    A           (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    A_egrl      (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    A_gdbk      (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    A_gulf      (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    A_labr      (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    A_nace      (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    ...          ...
    gulf        (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    labr        (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    gdbk        (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    nace        (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    egrl        (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>
    sum         (time, space) float64 dask.array<chunksize=(30, 730), meta=np.ndarray>

In [ ]:
# %%time
bins = 50
n = len(ds.time)
m = 1825#len(ds.space)
size = (n//bins+1, m//bins+1)
dics = {}
for var in rhs_list+['sl','sf']+region_names+['sum']:
    dics[var] = np.zeros(size)

j = 0
for start in range(0,n,bins):
    print(start, end =' ')

    xp = 0
    yp = start
    
    # dic = {}
    # for var in rhs_list+['sl','sf']:
    #     dic[var] = []
    i = 0
    while xp<=m and yp-bins>=0:
        
        sl = float(ds.sl[yp,xp])
        # print(xp, end = ' ')
        offset = np.minimum(m-xp,bins)
        sf = float(ds.sf[yp-offset+1,xp+offset-1])
        xslicer = xr.DataArray(np.arange(xp,xp+offset), dims = 'hold')
        yslicer = xr.DataArray(np.arange(yp,yp-offset,-1), dims = 'hold')
        
        dics['sl'][j,i] = sl
        dics['sf'][j,i] = sf
        for var in rhs_list+region_names+['sum']:
            cumsum = float(ds[var][yslicer,xslicer].sum())/NUMBER_OF_PARTICLE
            dics[var][j,i] = cumsum
        xp+=bins
        yp-=bins
        i+=1
    # for var in rhs_list+['sl','sf']:
    #     dic[var] = np.array(dic[var])
    # dics.append(dic)
    j+=1

0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 

In [ ]:
# %%time
covs = {}
for var in rhs_list+['sf','sl']:
    covs[var] = np.zeros(size[1])
for col in range(size[1]):
    print(col, end = ' ')
    var_contr = {}
    sl = dics['sl'][size[1]+1:,0]
    var_contr['sf'] = dics['sf'][size[1]+1:,col]
    length = len(sl)
    covs['sl'][col] = np.sum(sl*sl)/length
    for var in rhs_list:
        var_contr[var] = np.sum(dics[var][size[1]+1:,:col+1],axis = 1)
    for var in rhs_list+['sf']:
        dotpro = np.sum(var_contr[var]*sl)
        covs[var][col] = dotpro/length

In [ ]:
variable_dic = {'e_ua': 'sub-daily advection',
                'E': r"$u'\cdot \nabla s' - \overline{u'\cdot \nabla s'}$",
                'A': r"$u'\cdot \nabla \bar s$",
                'dif_h': 'horizontal diffusion',
                'dif_v': 'vertical diffusion',
                'I': 'Ice plume',
                'F': 'Evap/Prec',
                'sf': 'Initial salinity'
               }
xs = np.arange(0,size[1]*50+1,50)
for var in rhs_list+['sf']:
    if var == 'sf':
        plt.plot(xs,[1]+list(covs[var]/covs['sl']),label = variable_dic[var])
    else:
        plt.plot(xs,[0]+list(-covs[var]/covs['sl']),label = variable_dic[var],color = color_dic[var])
plt.xlabel('days before')
plt.ylabel(r'$\frac{EX_iY}{EY^2}$')
plt.legend(fontsize = 8)
# plt.savefig('covariance_tau.png')